In [2]:
# Importing nessecary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows

# 1. Load Data from Student.xlsx
df = pd.read_excel("Student.xlsx", sheet_name="Students")
subjects = ["Math", "Physics", "Chemistry", "Biology"]

# 2. Vectorized Computations
# Total score
df["Total"] = df[subjects].sum(axis=1)
# Average score
df["Average"] = df[subjects].mean(axis=1)

# Grade assigning 
df["Grade"] = np.select(
    [
        df["Average"] >= 90,
        (df["Average"] >= 75) & (df["Average"] < 90),
        (df["Average"] >= 60) & (df["Average"] < 75),
        df["Average"] < 60,
    ],
    ["A", "B", "C", "F"]
)

summary = df[["StudentID", "Name", "Total", "Average", "Grade"]]

# 3. Top Performers
top_dict = {subj: df.nlargest(3, subj)[["Name", subj]] for subj in subjects}

# 4. Create Bar graph using Matplotlib
# Average per subjects
avg_per_subject = df[subjects].mean()

plt.figure(figsize=(6,4))
plt.bar(avg_per_subject.index, avg_per_subject.values)
plt.title("Average Marks per Subject")
plt.xlabel("Subjects")
plt.ylabel("Marks")
plt.yticks(np.arange(0, 101, 25))  
plt.tight_layout()
plt.savefig("avg_subjects.png")
plt.close()

# 5. Write into Student.xlsx obtained results
wb = load_workbook("Student.xlsx")

# Summary sheet 
if "Summary" in wb.sheetnames:
    del wb["Summary"]
ws_summary = wb.create_sheet("Summary")

for r in dataframe_to_rows(summary, index=False, header=True):
    ws_summary.append(r)

# Top Performers sheet
if "Top Performers" in wb.sheetnames:
    del wb["Top Performers"]
ws_top = wb.create_sheet("Top Performers")

row = 1
for subject, data in top_dict.items():
    # Title row
    ws_top.cell(row=row, column=1, value=f"Top 3 in {subject}")
    row += 1

    # Data rows
    for name, score in zip(data["Name"], data[subject]):
        ws_top.cell(row=row, column=1, value=name)
        ws_top.cell(row=row, column=2, value=float(score))
        row += 1

    # Blank line before next subject
    row += 2

# --- Charts sheet ---
if "Charts" in wb.sheetnames:
    del wb["Charts"]
ws_chart = wb.create_sheet("Charts")

# Insert matplotlib chart image into Charts sheet
img = Image("avg_subjects.png")
ws_chart.add_image(img, "B2")

# Save workbook
wb.save("Student.xlsx")
